# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

# Paso 1: Importar librerías 

In [21]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns



# Paso 2: pedir permiso y descargar el HTML

In [22]:
url = "https://ycharts.com/companies/TSLA/revenues"

headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"}

response = requests.get(url, headers = headers)

if response.status_code ==200:
    contenido = response.content

else:
    print(f'status code distinto a 200: {response.status_code}') 

# Paso 3: Tranformar el HTML

In [23]:
soup = BeautifulSoup(contenido, 'html.parser')

# Las tablas tiene etiqueta table y clase table
tablas = soup.find_all('table', class_='table')

# La tabla de trimestrales esta dividida en 2 columnas
# tienen la siguiente estructura:
# <div class="row">
#     <div class="col-6">
#         <table class="table">
#             <thead>
#                 <tr>
#                     <th>Date</th>
#                     <th class="text-right">Value</th>
#                 </tr>
#             </thead>
#             <tbody>
                
#                     <tr>
#                         <td>June 30, 2024</td>
#                         <td class="text-right">
                            
#                                 25.50B
                            
#                         </td>
#                     </tr>

trimestrales = soup.find_all('div', class_='col-6')
for fila in trimestrales:
     date =fila.find_all('td')
     value= fila.find_all('td', class_='text-right')

# tablas = soup.find_all('table', class_='table')


In [24]:
trimestrales

[<div class="col-6">
 <table class="table">
 <thead>
 <tr>
 <th>Date</th>
 <th class="text-right">Value</th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td>June 30, 2024</td>
 <td class="text-right">
                             
                                 25.50B
                             
                         </td>
 </tr>
 <tr>
 <td>March 31, 2024</td>
 <td class="text-right">
                             
                                 21.30B
                             
                         </td>
 </tr>
 <tr>
 <td>December 31, 2023</td>
 <td class="text-right">
                             
                                 25.17B
                             
                         </td>
 </tr>
 <tr>
 <td>September 30, 2023</td>
 <td class="text-right">
                             
                                 23.35B
                             
                         </td>
 </tr>
 <tr>
 <td>June 30, 2023</td>
 <td class="text-right">
                            